# Wrangler Hello World

Execute the following command on Wrangler to start a Spark cluster (Not due to an issue with the SAGA SLURM adaptor it must be executed from the Home directory):


    saga-hadoop --resource=slurm://localhost --queue=normal --walltime=59 --number_cores=24 --project=TG-MCB090174 --framework spark

In [ ]:
!saga-hadoop --resource=slurm://localhost \
             --queue=normal --walltime=59 --number_cores=24 \
             --project=TG-MCB090174 --framework spark

### Optional: investigate environment on Wrangler

In [ ]:
!squeue

In [ ]:
!java -version

In [3]:
!/sbin/ifconfig

em1: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 10.1.58.118  netmask 255.255.255.0  broadcast 10.1.58.255
        ether f8:bc:12:55:dc:bc  txqueuelen 1000  (Ethernet)
        RX packets 20599091  bytes 9111907585 (8.4 GiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 602722  bytes 168991277 (161.1 MiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0
        device interrupt 98  

ib0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 2044
        inet 192.168.58.118  netmask 255.255.255.0  broadcast 192.168.58.255
Infiniband hardware address can be incorrect! Please read BUGS section in ifconfig(8).
        infiniband 80:00:02:40:FE:80:00:00:00:00:00:00:00:00:00:00:00:00:00:00  txqueuelen 256  (InfiniBand)
        RX packets 102777  bytes 5756850 (5.4 MiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 34  bytes 2040 (1.9 KiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

lo:

In [1]:
import os
os.environ["SPARK_LOCAL_IP"]="129.114.58.118"
%run ../util/init_spark_wrangler.py
from pilot_hadoop import PilotComputeService as PilotSparkComputeService


pilotcompute_description = {''
    "service_url": "spark://c251-141.wrangler.tacc.utexas.edu:7077",
    "spark.driver.host": os.environ["SPARK_LOCAL_IP"]
}

print "SPARK HOME: %s"%os.environ["SPARK_HOME"]
print "PYTHONPATH: %s"%os.environ["PYTHONPATH"]

start = time.time()
pilot_spark = PilotSparkComputeService.create_pilot(pilotcompute_description=pilotcompute_description)
sc = pilot_spark.get_spark_context()
print "Spark Startup, %.2f"%(time.time()-start)

SPARK Home: /home/01131/tg804093/work/spark-2.0.0-bin-hadoop2.6
SPARK HOME: /home/01131/tg804093/work/spark-2.0.0-bin-hadoop2.6
PYTHONPATH: /home/01131/tg804093/work/spark-2.0.0-bin-hadoop2.6/python:/home/01131/tg804093/work/spark-2.0.0-bin-hadoop2.6/python/lib/py4j-0.10.1-src.zip
Connect to Spark cluster: spark://c251-141.wrangler.tacc.utexas.edu:7077
spark.app.name=Pilot-Spark
spark.driver.extraClassPath="/home/01131/tg804093/spark-streaming-kafka-0-8_2.11-2.0.0.jar"
spark.driver.host=129.114.58.118
spark.executor.extraClassPath="/home/01131/tg804093/spark-streaming-kafka-0-8_2.11-2.0.0.jar"
spark.master=spark://c251-141.wrangler.tacc.utexas.edu:7077
spark.submit.deployMode=client
Spark Startup, 1.94


In [2]:
rdd=sc.parallelize(range(10))

In [3]:
rdd.count()

10

# Kafka

    saga-hadoop --resource=slurm://localhost    --queue=normal --walltime=180 --number_cores=32   --project=TG-MCB090174 --framework kafka

In [ ]:
!squeue -u tg804093

In [11]:
from pykafka import KafkaClient
zkKafka='c251-118.wrangler.tacc.utexas.edu:2181'

client = KafkaClient(zookeeper_hosts=zkKafka)
#client = KafkaClient(hosts='c251-142.wrangler.tacc.utexas.edu:9092')
topic = client.topics['test']
producer = topic.get_sync_producer()
consumer = topic.get_simple_consumer()

In [12]:
client.cluster.brokers

{0: <pykafka.broker.Broker at 0x2b2427139f90 (host=c251-119, port=9092, id=0)>,
 1: <pykafka.broker.Broker at 0x2b2427148450 (host=c251-118, port=9092, id=1)>}

In [20]:
producer.produce(str('Hello World dfasd dfads'))

In [18]:
message = consumer.consume(block=False)
print("%d: %s"%(message.offset, message.value))

1: Hello World 2


## Kafka and Spark Streaming

Kafka Integration only functional with spark-submit (see StreamingApp.py)

PySpark Init:

     /home/01131/tg804093/work/spark-2.0.0-bin-hadoop2.6/bin/pyspark --master spark://c251-121.wrangler.tacc.utexas.edu:7077 --packages  org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0

Download Spark Package:

    #!/bin/bash
    mvn dependency:get -Dartifact=org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0
    find $HOME/.m2/ -name "*.jar" |xargs cp -t /home/01131/tg804093/src/spark-streaming-kafka/

In [5]:
files=!ls /home/01131/tg804093/src/spark-streaming-kafka/
jar_files = ",".join(["/home/01131/tg804093/" + i for i in files])

jar_files_colon=":".join(["/home/01131/tg804093/" + i for i in files])

In [1]:
import os
#os.environ["PYSPARK_SUBMIT_ARGS"]="--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0"

%run util/init_spark_wrangler.py
from pilot_hadoop import PilotComputeService as PilotSparkComputeService

#os.environ["PYSPARK_SUBMIT_ARGS"]="--jars /home/01131/tg804093/src/spark-streaming-kafka/spark-streaming-kafka-0-10-assembly_2.11-2.0.0.jar"
#os.environ["SPARK_CLASSPATH"]=":".join(["/home/01131/tg804093/" + i for i in files])

#os.environ["SPARK_CLASSPATH"]="/home/01131/tg804093/spark-streaming-kafka-0-8_2.11-2.0.0.jar"


pilotcompute_description = {
    "service_url": "spark://c251-121.wrangler.tacc.utexas.edu:7077",
    #"spark.jars.packages": "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0",
    "spark.jars": "/home/01131/tg804093/spark-streaming-kafka-0-8_2.11-2.0.0.jar"
    #"spark.driver.host": os.environ["SPARK_LOCAL_IP"]
}

print "SPARK HOME: %s"%os.environ["SPARK_HOME"]
print "PYTHONPATH: %s"%os.environ["PYTHONPATH"]

start = time.time()
pilot_spark = PilotSparkComputeService.create_pilot(pilotcompute_description=pilotcompute_description)
sc = pilot_spark.get_spark_context()
print "Spark Startup, %.2f"%(time.time()-start)

SPARK Home: /home/01131/tg804093/work/spark-2.0.0-bin-hadoop2.6
SPARK HOME: /home/01131/tg804093/work/spark-2.0.0-bin-hadoop2.6
PYTHONPATH: /home/01131/tg804093/work/spark-2.0.0-bin-hadoop2.6/python:/home/01131/tg804093/work/spark-2.0.0-bin-hadoop2.6/python/lib/py4j-0.10.1-src.zip
Connect to Spark cluster: spark://c251-121.wrangler.tacc.utexas.edu:7077
spark.app.name=Pilot-Spark
spark.driver.extraClassPath="/home/01131/tg804093/spark-streaming-kafka-0-8_2.11-2.0.0.jar"
spark.executor.extraClassPath="/home/01131/tg804093/spark-streaming-kafka-0-8_2.11-2.0.0.jar"
spark.jars=/home/01131/tg804093/spark-streaming-kafka-0-8_2.11-2.0.0.jar
spark.master=spark://c251-121.wrangler.tacc.utexas.edu:7077
spark.submit.deployMode=client
Spark Startup, 1.81


In [2]:
zkKafka='c251-118.wrangler.tacc.utexas.edu:2181'
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
ssc = StreamingContext(sc, 1)
kvs = KafkaUtils.createStream(ssc, zkKafka, "spark-streaming-consumer", {'test': 1})    


________________________________________________________________________________________________

  Spark Streaming's Kafka libraries not found in class path. Try one of the following.

  1. Include the Kafka library and its dependencies with in the
     spark-submit command as

     $ bin/spark-submit --packages org.apache.spark:spark-streaming-kafka-0-8:2.0.0 ...

  2. Download the JAR of the artifact from Maven Central http://search.maven.org/,
     Group Id = org.apache.spark, Artifact Id = spark-streaming-kafka-0-8-assembly, Version = 2.0.0.
     Then, include the jar in the spark-submit command as

     $ bin/spark-submit --jars <spark-streaming-kafka-0-8-assembly.jar> ...

________________________________________________________________________________________________




TypeError: 'JavaPackage' object is not callable

In [ ]:
lines = kvs.map(lambda x: x[1])
counts = lines.flatMap(lambda line: line.split(" ")) \
        .map(lambda word: (word, 1)) \
        .reduceByKey(lambda a, b: a+b)
counts.pprint()
ssc.start()
ssc.awaitTermination()